In [ ]:
%%bash
rm -rf logdirs

In [ ]:
import tensorflow as tf
import FFBP

In [ ]:
NUM_EPOCHS = 500
INP_SIZE = (8,4)
TARG_SIZE = 36
DATA_LEN = 32
BATCH_SIZE = 4

FFBP_GRAPH = tf.Graph()

with FFBP_GRAPH.as_default():
    with tf.name_scope('train_data'):
        train_data = FFBP.InputData(
            path_to_data_file = 'eight_things-train_data.txt',
            num_epochs = NUM_EPOCHS,
            batch_size = BATCH_SIZE, 
            inp_size = INP_SIZE, 
            targ_size = TARG_SIZE,
            data_len = DATA_LEN,
            shuffle = True, 
            shuffle_seed = 1
        )

    with tf.name_scope('test_data'):
        test_data = FFBP.InputData(
            path_to_data_file = 'eight_things-train_data.txt',
            num_epochs = NUM_EPOCHS,
            batch_size = DATA_LEN,
            inp_size = INP_SIZE, 
            targ_size = TARG_SIZE,
            data_len = DATA_LEN,
            shuffle = False
        )

    model_name = 'eight_things_model'
    with tf.name_scope(model_name):

        item_inp  = tf.placeholder(dtype = tf.float32, shape=[None, INP_SIZE[0]], name='item_inp')
        relation_inp = tf.placeholder(dtype = tf.float32, shape=[None, INP_SIZE[1]], name='relation_inp')

        representation_layer = FFBP.BasicLayer(
            layer_name = 'representation_layer', 
            layer_input = item_inp, 
            size = 8, 
            wrange = [-.45, .45], 
            nonlin=tf.nn.sigmoid, 
            bias=True, 
            seed=1, # Use None for random seed value
        )

        hidden_layer = FFBP.BasicLayer(
            layer_name = 'hidden_layer', 
            layer_input = (representation_layer.output, relation_inp), 
            size = 12, 
            wrange = [-.45, .45], 
            nonlin=tf.nn.sigmoid, 
            bias=True, 
            seed=1, # Use None for random seed value
        )

        attribute_layer = FFBP.BasicLayer(
            layer_name = 'attribute_layer', 
            layer_input = hidden_layer.output, 
            size = 36, 
            wrange = [-.45, .45], 
            nonlin=tf.nn.sigmoid, 
            bias=True, 
            seed=1, # Use None for random seed value
        )

        target = tf.placeholder(dtype = tf.float32, shape=[None, TARG_SIZE], name='targets')

        MODEL = FFBP.Model(
            name = model_name,
            layers = [representation_layer, hidden_layer, attribute_layer],
            train_data = train_data, 
            inp        = [item_inp, relation_inp],
            targ       = target,
            loss       = tf.squared_difference(target, attribute_layer.output, name='loss_function'),
            optimizer  = tf.train.MomentumOptimizer(.1, 0),
            test_data = test_data
        )

In [ ]:
# Prevent unwanted logging messages by tensorflow (log ERROR messages only)
tf.logging.set_verbosity(tf.logging.ERROR) 

# Set up run parameters
NUM_RUNS = 1
TEST_EPOCHS = [i for i in range(0,800,100)]
SAVE_EPOCHS = [NUM_EPOCHS-1]
ECRIT = 0
RESTORE_DIR = None # provide path to logdir with a 'checkpoint_files' dir to restore a saved model


saver = FFBP.ModelSaver(restore_from=RESTORE_DIR, make_new_logdir=True)

for run_ind in range(NUM_RUNS):
    print('>>> RUN {}'.format(run_ind))
    
    with tf.Session(graph=FFBP_GRAPH) as sess:

        # restore or initialize FFBP_GRAPH variables:
        start_epoch = saver.init_model(session=sess)

        # create coordinator and start queue runners
        coordinator = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coordinator)

        for i in range(start_epoch, start_epoch + NUM_EPOCHS):
            # Test model occasionally
            if any([i==test_epoch for test_epoch in TEST_EPOCHS]):
                testloss, snap = MODEL.test_epoch(session=sess, verbose=True)
                saver.save_test(snap, run_ind)

            # Run one training epoch
            loss = MODEL.train_epoch(session=sess, verbose=False)
            saver.save_loss(loss, run_ind)

            # Save model occasionally
            if any([i==save_epoch for save_epoch in SAVE_EPOCHS]):
                saver.save_model(session=sess, model=MODEL)

            # Do final test, stop queues, and break out from training loop
            if loss < ECRIT or i == start_epoch + (NUM_EPOCHS - 1): 
                print('Final test ({})'.format(
                    'loss < ecrit' if loss < ECRIT else 'num_epochs reached'))

                testloss, snap = MODEL.test_epoch(session=sess, verbose=True)
                saver.save_test(snap, run_ind)

                coordinator.request_stop()
                coordinator.join(threads)

                saver.save_model(session=sess, model=MODEL)
                break